In [1]:
import pandas as pd
import numpy as np
import re
import time

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = 'https://datawrapper.dwcdn.net/X9xIP/8/'

In [3]:
## launch headless Edge instance
options = webdriver.EdgeOptions()
options.add_argument('--headless')

driver = webdriver.Edge(options = options)
driver.get(url)

## grab html from page
page = driver.page_source.encode('utf-8')
soup = BeautifulSoup(page, 'html.parser')

In [4]:
page_count = soup.find('div', {'class': re.compile(r'pagination')}).text
page_count = int(re.findall(r'(?<=Page 1 of )\d+', page_count)[0])

In [5]:
rows = []

for count in range(page_count-1):
    
    page = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(page, 'html.parser')
    tbody = soup.find_all('tbody')[0]
    
    for child in tbody.children:
        if child == ' ':
            break
        row = []
        for td in child:
            row.append(td.text)
        rows.append(row)
    
    if count == page_count:
        break
    
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((
            By.XPATH, 
            "//div[@class='pagination export-hide datawrapper-X9xIP-xyhjtd svelte-1ya2siw']//button[@class='next svelte-1ya2siw']"))).click()

In [6]:
pvi_df = pd.DataFrame(rows, 
                      columns = ['null', 'district', 'name', 'party',
                                 'district_description', 'pvi', 'biden',
                                 'trump', 'clinton', 'trump'])

In [7]:
pvi_df

,null,district,name,party,district_description,pvi,biden,trump,clinton,trump
0,,AL-01,Jerry Carl,R,Southwest: Mobile,R+16,35.2%,63.5%,33.8%,62.9%
1,,AL-02,Barry Moore,R,"Southeast: Dothan, the",R+17,34.7%,64.1%,32.3%,64.8%
2,,AL-03,Mike Rogers,R,"East: Auburn, Anniston, Talladega",R+19,32.4%,66.4%,31.1%,65.7%
3,,AL-04,Robert Aderholt,R,"North central: Cullman, Fort Payne",R+33,18.6%,80.2%,18.0%,79.0%
4,,AL-05,OPEN (Brooks),R,North: Huntsville,R+17,35.5%,62.5%,30.8%,63.8%
...,...,...,...,...,...,...,...,...,...,...
420,,WA-07,Pramila Jayapal,D,Seattle,D+36,86.3%,11.2%,83.4%,11.0%
421,,WA-08,Kim Schrier,D,"Seattle exurbs: Issaquah, Sammamish, Wenatchee",D+1,51.7%,45.0%,47.7%,44.5%
422,,WA-09,Adam Smith,D,"Seattle suburbs: Renton, Kent, Auburn",D+21,71.0%,26.3%,68.7%,24.9%
423,,WA-10,Marilyn Strickland,D,"Olympia, Puyallup, parts of Tacoma",D+7,56.9%,39.3%,52.2%,39.1%


In [9]:
pvi_df.drop(columns = 'null').to_csv('data/cook_pvi.csv', index = False)